In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pydelfi.priors as priors
import pydelfi.ndes as ndes
import pydelfi.delfi as delfi
import tensorflow as tf
import pickle

# prepare the data

In [ ]:
#define the parameter prior 
lower = np.array([4,1])
upper = np.array([6,2.398])
prior = priors.Uniform(lower, upper)

#validation data
theta_fiducials = np.load('data/hera_para_validation.npy')
compressed_datas = np.load('data/hera_ps_validation.npy')

#training data
sim_params = np.load('data/hera_para.npy')[0:9000]
sim_compressed_data=np.load('data/hera_ps.npy')[0:9000]

#dimension of the data summary
nd=compressed_datas.shape[1]

#0 for the faint model; 1 for the bright model
theta_fiducial = theta_fiducials[i]
compressed_data = compressed_datas[i]

#the directory to load the trained model
diri = "data/model/trained_hera/"

# define the function for validation on a single mock observation

In [ ]:
def pos(i):
   
    theta_fiducial = theta_fiducials[i]
    compressed_data = compressed_datas[i]

    NDEs = [ndes.ConditionalMaskedAutoregressiveFlow(n_parameters=2, n_data=nd, n_hiddens=[50,50], n_mades=5, act_fun=tf.tanh, index=0),
            ndes.ConditionalMaskedAutoregressiveFlow(n_parameters=2, n_data=nd, n_hiddens=[50,50], n_mades=5, act_fun=tf.tanh, index=1),
            ndes.ConditionalMaskedAutoregressiveFlow(n_parameters=2, n_data=nd, n_hiddens=[50,50], n_mades=5, act_fun=tf.tanh, index=2),
            ndes.ConditionalMaskedAutoregressiveFlow(n_parameters=2, n_data=nd, n_hiddens=[50,50], n_mades=5, act_fun=tf.tanh, index=3)]
    

    DelfiEnsemble = delfi.Delfi(compressed_data, prior, NDEs, 
                            Finv = None,
                            n_procs = 1,
                            theta_fiducial = theta_fiducial, 
                            param_limits = [lower, upper],
                            param_names = ['\mathrm{log_{10}\,T_{vir}}', '\mathrm{log_{10}\,\zeta}'], 
                            results_dir = diri,
                            progress_bar = False,
                            save = False,
                            restore = True,
                            input_normalization=None)

    
    posterior_samples = DelfiEnsemble.emcee_sample(burn_in_chain=600, main_chain=1000)
    with open('data/validate/po_validate_%d.pkl'%i, 'wb') as f:
        pickle.dump(posterior_samples,f)
  

# validate on a pre-set number of observations

In [ ]:
#can validate on different mock observations; better to run in parallel
#for i in range(len(compressed_datas)):
for i in range(1):
    pos(i)